In [1]:
import numpy as np
print(np.__version__)

1.18.5


# Pre-processing Steps

<ol>1. Filling the missing values</ol>
<ol>2. Converting categories to numbers</ol>
<ol>3. Bring all the variables in range 0 to 1</ol>

In [2]:
# importing required libraries
import pandas as pd
# check version on pandas
print('Version of pandas:', pd.__version__)  

Version of pandas: 1.0.5


In [3]:
# reading the loan prediction data
train = pd.read_csv('train_XnW6LSF.csv')
test = pd.read_csv('test_FewQE9B.csv')
sample_sub = pd.read_csv('sample_submission_hP4II7x.csv')

In [4]:
data = train.copy()
print("Shape", data.shape)
# checking missing values in the data
print("Nulls",round((data.isnull().sum()/len(data))*100,2) )
print("DataTypes")
# data types of the variables
print(data.dtypes)
data.head()

Shape (8523, 12)
Nulls Item_Identifier               0.00
Item_Weight                  17.17
Item_Fat_Content              0.00
Item_Visibility               0.00
Item_Type                     0.00
Item_MRP                      0.00
Outlet_Identifier             0.00
Outlet_Establishment_Year     0.00
Outlet_Size                  28.28
Outlet_Location_Type          0.00
Outlet_Type                   0.00
Item_Outlet_Sales             0.00
dtype: float64
DataTypes
Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object


,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [5]:
#Multiple records of loanids?
data.groupby(['Item_Identifier','Outlet_Identifier']).size().sort_values(ascending=False).head() #No

Item_Identifier  Outlet_Identifier
NCZ54            OUT049               1
FDJ27            OUT049               1
FDJ22            OUT017               1
                 OUT018               1
                 OUT019               1
dtype: int64

## 1. Filling the missing values

### Categorical Data: Mode

In [6]:

data['Outlet_Size'].fillna(data['Outlet_Size'].mode()[0], inplace=True)

In [7]:
# data.isnull().sum()

### Continuous Data: Mean

In [8]:
# filling missing values of continuous variables with mean
data['Item_Weight'].fillna(data['Item_Weight'].mean(), inplace=True)

In [9]:
# checking missing values after imputation
print("Nulls",round((data.isnull().sum()/len(data))*100,2) )

Nulls Item_Identifier              0.0
Item_Weight                  0.0
Item_Fat_Content             0.0
Item_Visibility              0.0
Item_Type                    0.0
Item_MRP                     0.0
Outlet_Identifier            0.0
Outlet_Establishment_Year    0.0
Outlet_Size                  0.0
Outlet_Location_Type         0.0
Outlet_Type                  0.0
Item_Outlet_Sales            0.0
dtype: float64


## 2. Converting categories to numbers

In [10]:
data.Item_Fat_Content.unique()

array(['Low Fat', 'Regular', 'low fat', 'LF', 'reg'], dtype=object)

In [11]:
cat_cols = data.select_dtypes(include=['object']).columns.tolist()
print(cat_cols)
cat_cols.remove('Item_Identifier')
cat_cols.remove('Outlet_Identifier')
cat_cols.remove('Item_Fat_Content')
cat_cols

['Item_Identifier', 'Item_Fat_Content', 'Item_Type', 'Outlet_Identifier', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']


['Item_Type', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']

In [12]:
# converting the categories into numbers using map function
dict1 = {'Low Fat': 0, 'low fat': 0, 'LF':0, 'Regular':1, 'reg': 1}
data['Item_Fat_Content'] = data['Item_Fat_Content'].map(dict1)
data.Item_Fat_Content.unique()

array([0, 1])

In [13]:
data_item_id = data["Item_Identifier"].copy()
data_outlet_id = data["Outlet_Identifier"].copy()
data_target = data['Item_Outlet_Sales'].copy()
model_df = data.drop(["Item_Identifier","Outlet_Identifier","Item_Outlet_Sales"], axis = 1)
num_cols = model_df.select_dtypes(include=['float','int']).columns.tolist()
num_cols
# Convert categorical variables to numeric - one hot encoder

from sklearn.preprocessing import OneHotEncoder


cat_encoder = OneHotEncoder(sparse=False)
model_cat = model_df[cat_cols]
model_cat_1hot = cat_encoder.fit_transform(model_cat)
# model_cat_1hot.toarray()

#     print(cat_encoder.categories_)


# Scale numeric
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('std_scaler', StandardScaler())
])




model_num = model_df.drop(cat_cols, axis = 1)
model_num_tr = num_pipeline.fit_transform(model_num)


from sklearn.compose import ColumnTransformer

num_attribs = list(model_num)
cat_attribs = cat_cols

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', OneHotEncoder(sparse=False), cat_attribs)
])


model_prepared = full_pipeline.fit_transform(model_df)



In [14]:
trans_df = pd.DataFrame(
    model_prepared,
    columns=num_cols + cat_encoder.get_feature_names().tolist())

trans_df["Item_Identifier"] = data_item_id
trans_df["Outlet_Identifier"] = data_outlet_id
trans_df["Item_Outlet_Sales"] = data_target
cols = list(trans_df)
cols.insert(0, cols.pop(cols.index('Outlet_Identifier')))
cols.insert(0, cols.pop(cols.index('Item_Identifier')))
trans_df = trans_df.loc[:, cols]
trans_df.head()

,Item_Identifier,Outlet_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_MRP,Outlet_Establishment_Year,x0_Baking Goods,x0_Breads,x0_Breakfast,x0_Canned,x0_Dairy,x0_Frozen Foods,x0_Fruits and Vegetables,x0_Hard Drinks,x0_Health and Hygiene,x0_Household,x0_Meat,x0_Others,x0_Seafood,x0_Snack Foods,x0_Soft Drinks,x0_Starchy Foods,x1_High,x1_Medium,x1_Small,x2_Tier 1,x2_Tier 2,x2_Tier 3,x3_Grocery Store,x3_Supermarket Type1,x3_Supermarket Type2,x3_Supermarket Type3,Item_Outlet_Sales
0,FDA15,OUT049,-0.841872,-0.738147,-0.970732,1.747454,0.139541,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,3735.1380
1,DRC01,OUT018,-1.641706,1.354743,-0.908111,-1.489023,1.334103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,443.4228
2,FDN15,OUT049,1.098554,-0.738147,-0.956917,0.010040,0.139541,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2097.2700
3,FDX07,OUT010,1.500838,1.354743,-1.281758,0.660050,0.020085,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,732.3800
4,NCD19,OUT013,-0.929428,-0.738147,-1.281758,-1.399220,-1.293934,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,994.7052


In [15]:
# saving the pre-processed data
trans_df.to_csv('bigmart_processed_data.csv', index=False)

#Solving_Loan_Prediction_problem_using_Neural_Network

In [16]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


# Steps to build a Neural Network using Keras

<ol>1. Loading the dataset</ol>
<ol>2. Creating training and validation set</ol>
<ol>3. Defining the architecture of the model</ol>
<ol>4. Compiling the model (defining loss function, optimizer)</ol>
<ol>5. Training the model</ol>
<ol>6. Evaluating model performance on training and validation set</ol>

## 1. Loading the dataset

In [17]:
# importing the required libraries
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
%matplotlib inline

In [18]:
# check version on sklearn
print('Version of sklearn:', sklearn.__version__)

Version of sklearn: 0.22.2.post1


In [19]:
# loading the pre-processed dataset
data = pd.read_csv('bigmart_processed_data.csv')

In [20]:
# looking at the first five rows of the dataset
data.head()

,Item_Identifier,Outlet_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_MRP,Outlet_Establishment_Year,x0_Baking Goods,x0_Breads,x0_Breakfast,x0_Canned,x0_Dairy,x0_Frozen Foods,x0_Fruits and Vegetables,x0_Hard Drinks,x0_Health and Hygiene,x0_Household,x0_Meat,x0_Others,x0_Seafood,x0_Snack Foods,x0_Soft Drinks,x0_Starchy Foods,x1_High,x1_Medium,x1_Small,x2_Tier 1,x2_Tier 2,x2_Tier 3,x3_Grocery Store,x3_Supermarket Type1,x3_Supermarket Type2,x3_Supermarket Type3,Item_Outlet_Sales
0,FDA15,OUT049,-0.841872,-0.738147,-0.970732,1.747454,0.139541,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,3735.1380
1,DRC01,OUT018,-1.641706,1.354743,-0.908111,-1.489023,1.334103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,443.4228
2,FDN15,OUT049,1.098554,-0.738147,-0.956917,0.010040,0.139541,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2097.2700
3,FDX07,OUT010,1.500838,1.354743,-1.281758,0.660050,0.020085,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,732.3800
4,NCD19,OUT013,-0.929428,-0.738147,-1.281758,-1.399220,-1.293934,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,994.7052


In [21]:

# checking missing values
data.isnull().sum()

Item_Identifier              0
Outlet_Identifier            0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_MRP                     0
Outlet_Establishment_Year    0
x0_Baking Goods              0
x0_Breads                    0
x0_Breakfast                 0
x0_Canned                    0
x0_Dairy                     0
x0_Frozen Foods              0
x0_Fruits and Vegetables     0
x0_Hard Drinks               0
x0_Health and Hygiene        0
x0_Household                 0
x0_Meat                      0
x0_Others                    0
x0_Seafood                   0
x0_Snack Foods               0
x0_Soft Drinks               0
x0_Starchy Foods             0
x1_High                      0
x1_Medium                    0
x1_Small                     0
x2_Tier 1                    0
x2_Tier 2                    0
x2_Tier 3                    0
x3_Grocery Store             0
x3_Supermarket Type1         0
x3_Supermarket Type2         0
x3_Super

In [22]:
# checking the data type
data.dtypes

Item_Identifier               object
Outlet_Identifier             object
Item_Weight                  float64
Item_Fat_Content             float64
Item_Visibility              float64
Item_MRP                     float64
Outlet_Establishment_Year    float64
x0_Baking Goods              float64
x0_Breads                    float64
x0_Breakfast                 float64
x0_Canned                    float64
x0_Dairy                     float64
x0_Frozen Foods              float64
x0_Fruits and Vegetables     float64
x0_Hard Drinks               float64
x0_Health and Hygiene        float64
x0_Household                 float64
x0_Meat                      float64
x0_Others                    float64
x0_Seafood                   float64
x0_Snack Foods               float64
x0_Soft Drinks               float64
x0_Starchy Foods             float64
x1_High                      float64
x1_Medium                    float64
x1_Small                     float64
x2_Tier 1                    float64
x

In [23]:
# removing the loan_ID since these are just the unique values
data_item_id = data["Item_Identifier"].copy()
data_outlet_id = data["Outlet_Identifier"].copy()
data = data.drop(["Item_Identifier","Outlet_Identifier"], axis = 1)

In [24]:
# looking at the shape of the data
data.shape

(8523, 32)

In [25]:
# separating the independent and dependent variables

# storing all the independent variables as X
X = data.drop('Item_Outlet_Sales', axis=1)

# storing the dependent variable as y
y = data['Item_Outlet_Sales']

In [26]:
# shape of independent and dependent variables
X.shape, y.shape

((8523, 31), (8523,))

## 2. Creating training and validation set

In [27]:
# Creating training and validation set

# random state to regenerate the same train and validation set
# test size 0.2 will keep 20% data in validation and remaining 80% in train set

X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=10,test_size=0.2)

In [28]:
# shape of training and validation set
(X_train.shape, y_train.shape), (X_test.shape, y_test.shape)

(((6818, 31), (6818,)), ((1705, 31), (1705,)))

## 3. Defining the architecture of the model

In [29]:
# checking the version of keras
import keras
print(keras.__version__)

Using TensorFlow backend.


2.3.1


In [30]:
# checking the version of tensorflow
import tensorflow as tf
print(tf.__version__)

1.15.2


### a. Create a model

<img src='https://drive.google.com/uc?id=1iZNZ3kwSHRNf-Irn3DZmMuBb6K-Lro7w'>

### b. Defining different layers

<img src='https://drive.google.com/uc?id=16X6De2hua1XJBe3dfmUUeGTgP6PbXEpc'>

<img src='https://drive.google.com/uc?id=1tsy4B6G0UN4-J4L4roOdoWQiZMUdgw2a'>

In [31]:
# number of input neurons
X_train.shape


(6818, 31)

In [32]:
# number of features in the data
X_train.shape[1]

31

In [33]:
# defining input neurons
input_neurons = X_train.shape[1]

<img src='https://drive.google.com/uc?id=1xL_hM9rGItZjsZ8Lofwzw_9fZUi4bgJo'>

In [ ]:
# number of output neurons

# since loan prediction is a binary classification problem, we will have single neuron in the output layer 

In [34]:
# define number of output neurons
output_neurons = 1

In [35]:
# number of hidden layers and hidden neurons

# It is a hyperparameter and we can pick the hidden layers and hidden neurons on our own

In [36]:
# # define hidden layers and neuron in each layer
# number_of_hidden_layers = 2
# neuron_hidden_layer_1 = 20
# neuron_hidden_layer_2 = 10

In [37]:
#https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/
# define base model
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(input_neurons, input_dim=input_neurons, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

In [38]:
from keras.wrappers.scikit_learn import KerasRegressor
# importing the sequential model
from keras.models import Sequential
# importing different layers from keras
from keras.layers import InputLayer, Dense 

estimator = KerasRegressor(build_fn=baseline_model, epochs=30, batch_size=5, verbose=0)

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5)
results = cross_val_score(estimator, X_train, y_train, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Results: -1233968.12 (48680.10) MSE


In [ ]:
# Results: -1192392.75 (80768.37) MSE

In [ ]:
# model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50)

## 6. Evaluating model performance on validation set

In [39]:

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


In [40]:
estimator.fit(X_train, y_train)
prediction = estimator.predict(X_test)
mean_absolute_percentage_error(y_test, prediction) #54.58289499388174

59.433672005054426

In [43]:
# Save the entire model as a SavedModel.
!mkdir -p saved_model
estimator.model.save('saved_model/my_model') 

In [46]:
from keras.models import load_model

# Instantiate the model as you please (we are not going to use this)
model2 = KerasRegressor(build_fn=baseline_model, epochs=10, batch_size=10, verbose=1)

# This is where you load the actual saved model into new variable.
model2.model = load_model('saved_model/my_model')

# Now you can use this to predict on new data (without fitting model2, because it uses the older saved model)
model2.model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 31)                992       
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 32        
Total params: 1,024
Trainable params: 1,024
Non-trainable params: 0
_________________________________________________________________


# Predicting on Test

In [47]:
test = pd.read_csv('test_FewQE9B.csv')
print(test.shape)

print(test.isnull().sum())
test['Outlet_Size'].fillna(test['Outlet_Size'].mode()[0], inplace=True)
# filling missing values of continuous variables with mean
test['Item_Weight'].fillna(test['Item_Weight'].mean(), inplace=True)
cat_cols = test.select_dtypes(include=['object']).columns.tolist()
# print(cat_cols)
cat_cols.remove('Item_Identifier')
cat_cols.remove('Outlet_Identifier')
cat_cols.remove('Item_Fat_Content')
print(cat_cols)
# converting the categories into numbers using map function
dict1 = {'Low Fat': 0, 'low fat': 0, 'LF':0, 'Regular':1, 'reg': 1}
test['Item_Fat_Content'] = test['Item_Fat_Content'].map(dict1)
test_num_cols = test.select_dtypes(include=['float','int']).columns.tolist()
print(test_num_cols)
test_item_id = test["Item_Identifier"].copy()
test_outlet_id = test["Outlet_Identifier"].copy()
model_df = test.drop(["Item_Identifier","Outlet_Identifier"], axis = 1)

print(model_df.shape)
model_df.head(2)

(5681, 11)
Item_Identifier                 0
Item_Weight                   976
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  1606
Outlet_Location_Type            0
Outlet_Type                     0
dtype: int64
['Item_Type', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']
['Item_Weight', 'Item_Fat_Content', 'Item_Visibility', 'Item_MRP', 'Outlet_Establishment_Year']
(5681, 9)


,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,20.75,0,0.007565,Snack Foods,107.8622,1999,Medium,Tier 1,Supermarket Type1
1,8.30,1,0.038428,Dairy,87.3198,2007,Medium,Tier 2,Supermarket Type1


In [48]:
cat_cols

['Item_Type', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']

In [49]:


# Convert categorical variables to numeric - one hot encoder

from sklearn.preprocessing import OneHotEncoder


test_cat_encoder = OneHotEncoder(sparse=False)
model_cat = model_df[cat_cols]

model_cat_1hot = test_cat_encoder.fit_transform(model_cat)

In [50]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('std_scaler', StandardScaler())
])

In [51]:
model_num = model_df.drop(cat_cols, axis = 1)
model_num_tr = num_pipeline.fit_transform(model_num)


In [52]:
model_num_tr.shape

(5681, 5)

In [53]:
from sklearn.compose import ColumnTransformer

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, test_num_cols),
    ('cat', OneHotEncoder(sparse=False), cat_cols)
])

In [54]:
test_model_prepared = full_pipeline.fit_transform(model_df)

In [55]:
len(test_num_cols) + len(test_cat_encoder.get_feature_names().tolist()) 

31

In [56]:
test_trans_df = pd.DataFrame(
    test_model_prepared,
    columns=test_num_cols + test_cat_encoder.get_feature_names().tolist())

test_trans_df.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_MRP,Outlet_Establishment_Year,x0_Baking Goods,x0_Breads,x0_Breakfast,x0_Canned,x0_Dairy,x0_Frozen Foods,x0_Fruits and Vegetables,x0_Hard Drinks,x0_Health and Hygiene,x0_Household,x0_Meat,x0_Others,x0_Seafood,x0_Snack Foods,x0_Soft Drinks,x0_Starchy Foods,x1_High,x1_Medium,x1_Small,x2_Tier 1,x2_Tier 2,x2_Tier 3,x3_Grocery Store,x3_Supermarket Type1,x3_Supermarket Type2,x3_Supermarket Type3
0,1.897460e+00,-0.740811,-1.134077,-0.536555,0.139891,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,-1.035530e+00,1.349873,-0.531850,-0.868937,1.095512,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,4.486336e-01,-0.740811,0.661316,1.629848,0.020438,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,-1.267578e+00,-0.740811,-0.981416,0.226697,1.095512,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,-5.230960e-14,1.349873,1.032540,1.508110,-1.532446,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [ ]:
# test_trans_df["Item_Identifier"] = test_item_id
# test_trans_df["Outlet_Identifier"] = test_outlet_id
# cols = list(test_trans_df)
# cols.insert(0, cols.pop(cols.index('Outlet_Identifier')))
# cols.insert(0, cols.pop(cols.index('Item_Identifier')))
# test_trans_df = test_trans_df.loc[:, cols]
# test_trans_df.head()

# Predicting on test

In [57]:
estimator.fit(X, y)
prediction = estimator.predict(test_trans_df)
prediction

array([1753.051 , 1380.8638, 1535.0352, ..., 1995.6058, 3620.0154,
       1409.0858], dtype=float32)

In [59]:
sample_sub.head()

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,1000
1,FDW14,OUT017,1000
2,NCN55,OUT010,1000
3,FDQ58,OUT017,1000
4,FDY38,OUT027,1000


In [60]:
sub1 = pd.DataFrame(prediction, columns= ['Item_Outlet_Sales'])
sub1["Item_Identifier"] = test_item_id
sub1["Outlet_Identifier"] = test_outlet_id
sub1.head()

,Item_Outlet_Sales,Item_Identifier,Outlet_Identifier
0,1753.051025,FDW58,OUT049
1,1380.863770,FDW14,OUT017
2,1535.035156,NCN55,OUT010
3,2540.588623,FDQ58,OUT017
4,5136.916504,FDY38,OUT027


In [61]:
sub1 = sub1[['Item_Identifier','Outlet_Identifier','Item_Outlet_Sales']]
# saving the predicted data
sub1.to_csv('bigmart_sub1.csv', index=False)

In [62]:
sub1.head()

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,1753.051025
1,FDW14,OUT017,1380.863770
2,NCN55,OUT010,1535.035156
3,FDQ58,OUT017,2540.588623
4,FDY38,OUT027,5136.916504


In [63]:
sub = pd.read_csv('bigmart_sub1.csv')

In [69]:
sub1.describe()

,Item_Outlet_Sales
count,5681.000000
mean,2324.582764
std,1154.143799
min,1.018364
25%,1509.161621
50%,2167.426270
75%,3105.457031
max,5787.524902


In [67]:
impute_mean = sub1["Item_Outlet_Sales"].mean()

In [68]:
sub1["Item_Outlet_Sales"] = np.where(sub1["Item_Outlet_Sales"] <0, impute_mean, sub1["Item_Outlet_Sales"])

In [70]:
sub1.to_csv('bigmart_sub1.csv', index=False)

In [71]:
sub = pd.read_csv('bigmart_sub1.csv')